# Lab 10: A simple Agent with Tools

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/DSML4220/blob/main/lab10_agents_and_tools.ipynb)

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/DSML4220/blob/main/lab10_agents_and_tools.ipynb)

In this lab we will use Ollama to create a simple agent armed with tools in order to help carry out tasks on our behalf. 


### Lab 10 Assignment/Task
There are a few questions below that require some additional code to be written so that your agent can carry out other operations besides just addition. 

Let's start out by setting up Ollama to run in Colab. If you run this notebook locally and already have Ollama running, then you can skip these steps. 

In [ ]:
! do some stuff

Now that Ollama is running, we can get started. The only module/library needed for this is the ollama-python module.

In [3]:
import ollama

In [46]:
# Tool function to add two numbers
def add_two_numbers(a: int, b: int) -> int:
    return a + b

In [47]:
# System prompt to inform the model about the tool is usage
system_message = {
    "role": "system", 
    "content": "You are a helpful assistant. You can do math by calling a function 'add_two_numbers' if needed."
}


# A sample of user input asking a math question
user_message = {
    "role": "user", 
    #"content": "What is 10 + 10?"
    #"content": "What is 135 * 4?"
    "content": "What is 90999999 + 10000001?"
}

messages = [system_message, user_message]

messages

[{'role': 'system',
  'content': "You are a helpful assistant. You can do math by calling a function 'add_two_numbers' if needed."},
 {'role': 'user', 'content': 'What is 90999999 + 10000001?'}]

In [48]:
# Ask llama3.2 to respond 
response = ollama.chat(
    model='llama3.2:1b', 
    messages=messages,
    tools=[add_two_numbers]#, multiply_two_numbers]  # pass the actual function object as a tool
)

In [49]:
response.message

Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='add_two_numbers', arguments={'a': '90999999', 'b': '10000001'}))])

In [50]:
response.message.content

''

In [51]:
# Check if the model called a function
if response.message.tool_calls:
    for tool_call in response.message.tool_calls:
        func_name = tool_call.function.name   # e.g., "add_two_numbers"
        args = tool_call.function.arguments   # e.g., {"a": 10, "b": 10}
        # If the function name matches and we have it in our tools, execute it:
        if func_name == "add_two_numbers":
            result = add_two_numbers(**args)
            print("Function output:", result)




Function output: 9099999910000001


---

### Q1: Does the above output look correct? Does it look like the sum of the numbers 90999999 and 10000001? Why is it not correct?

(Hint: there is nothing wrong with the model/agent here, but rather the tool implementation; namely, Python's [type hints](https://docs.python.org/3/library/typing.html) are not a guarantee that the correct/intended data type is used, so you may need to add some type casting inside of the function `add_two_numbers`)

`<INSERT YOUR ANSWER HERE>`

---

In [36]:


# putting it all together
""" (Continuing from previous code) """
available_functions = {"add_two_numbers": add_two_numbers}#, "multiply_two_numbers": multiply_two_numbers}

""" System prompt to inform the model about the tool is usage """

""" Model's initial response after possibly invoking the tool """
assistant_reply = response.message.content
print("Assistant (initial):", assistant_reply)

""" If a tool was called, handle it """
for tool_call in (response.message.tool_calls or []):
    func = available_functions.get(tool_call.function.name)
    if func:
        result = func(**tool_call.function.arguments)
        # Provide the result back to the model in a follow-up message
        messages.append({"role": "assistant", "content": f"The result is {result}."})
        follow_up = ollama.chat(model='llama3.2', messages=messages)
        print("Assistant (final):", follow_up.message.content)

Assistant (initial): 
Assistant (final):  

Here's how I calculated it:

I couldn't find the exact formula to add these two numbers, so I used addition manually:
 
90999999
+ 10000001
------
9099999910000000


In [52]:
from langchain_community.tools import DuckDuckGoSearchResults


def search_web(query: str) -> str:
  return DuckDuckGoSearchResults(backend="news").run(query)


tool_search_web = {'type':'function', 'function':{
  'name': 'search_web',
  'description': 'Search the web',
  'parameters': {'type': 'object',
                'required': ['query'],
                'properties': {
                    'query': {'type':'str', 'description':'the topic or subject to search on the web'},
}}}}
## test
search_web(query="nvidia")

ImportError: Could not import duckduckgo-search python package. Please install it with `pip install -U duckduckgo-search`.

In [ ]:
def search_yf(query: str) -> str:
  engine = DuckDuckGoSearchResults(backend="news")
  return engine.run(f"site:finance.yahoo.com {query}")

tool_search_yf = {'type':'function', 'function':{
  'name': 'search_yf',
  'description': 'Search for specific financial news',
  'parameters': {'type': 'object',
                'required': ['query'],
                'properties': {
                    'query': {'type':'str', 'description':'the financial topic or subject to search'},
}}}}

## test
search_yf(query="nvidia")

---

### Q3: How many fewer parameters did the LoRA model need to train/tune than the full GPT-2 model did?

(Hint: See output from above cell)

`<INSERT YOUR ANSWER HERE>`

---